In [28]:
##%%
#import pandas as pd
import numpy as np
import re
import os


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
sns.set_theme(style='white')
import matplotlib.dates as mdates

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import random

In [3]:
from sqlalchemy import create_engine, text

def postgresql_engine(user, pwd, host, port, dbname):
    # Need psycopg2-binary package
    sql_engine = create_engine('postgres://' + user + ':' + pwd + '@' + host + ':' + port + '/' + dbname, echo=False)
    return sql_engine

In [4]:
# DB username & password
import getpass

username = getpass.getpass()
password = getpass.getpass()

In [7]:
# misc db parameters
url= 'adds-postgres-dev.cfgztrijqgvp.us-east-1.rds.amazonaws.com'
database= 'musiclab'
port= '5432'

In [10]:
data_query = '''
Select *
from adds_temp.ebw_features_h1 as efh
'''

In [11]:
engine = postgresql_engine(username, password, url, port, database)
with engine.connect() as conn:
    with conn.begin():
        df_ebw_features= pd.read_sql(data_query, con=conn)

### Calculated fields for analysis

In [12]:
# Look at two appearances in top quintile of callout research
df_pop_quintile = pd.DataFrame(df_ebw_features[~pd.isna(df_ebw_features['pop'])].groupby(['format_code', 'cmm_station_calls', 'week_dt']).apply(lambda x: np.quantile(x['pop'], 0.80)), columns=['top_quintile_cutoff'])

In [13]:
df_pop_quintile

top_quintile_cutoff
format_code cmm_station_calls week_dt                        
H1          KBKS-FM           2018-11-11                 58.2
                              2018-11-25                 68.0
                              2019-01-06                 73.8
                              2019-01-20                 75.6
                              2019-02-03                 79.0
...                                                       ...
            WZFT-FM           2022-10-23                 86.4
                              2022-11-06                 83.4
                              2022-12-04                 78.0
                              2023-01-01                 87.6
                              2023-01-22                 80.6

[2939 rows x 1 columns]

In [14]:
df_ebw_features['is_top_quintile'] = df_ebw_features.join(df_pop_quintile, on=['format_code', 'cmm_station_calls', 'week_dt'], rsuffix='_r').apply(lambda x: int(x['pop'] >= x['top_quintile_cutoff']), axis=1)

In [15]:
df_ebw_features['week_dt'] = pd.to_datetime(df_ebw_features['week_dt'])

In [16]:
df_ebw_features.sort_values(by=['format_code', 'call_letters', 'mediabase_id', 'week_dt'], inplace=True)

In [17]:
df_ebw_features['cuml_spins_non_on'] = df_ebw_features.groupby(['format_code', 'call_letters', 'mediabase_id'])['spins_non_on'].cumsum()

In [18]:
df_ebw_features['num_top_quintile'] = df_ebw_features[df_ebw_features['cuml_spins_non_on'] >= 50].groupby(['format_code', 'call_letters', 'mediabase_id'])['is_top_quintile'].cumsum()

In [19]:
100*(int(np.max(df_ebw_features['cuml_spins_non_on'])/100) + 1)

6400

In [20]:
df_ebw_features['cuml_spins_bucket'] = pd.cut(df_ebw_features['cuml_spins_non_on'], bins=pd.interval_range(start=0, end=100*(int(np.max(df_ebw_features['cuml_spins_non_on'])/100) + 1), freq=100))

In [21]:
df_ebw_features['weeks_since_release'] = ((df_ebw_features['week_dt'] - pd.to_datetime(df_ebw_features['song_release_date']))/np.timedelta64(1, 'W')).apply(int)

In [23]:
ddl_range = [20]

In [24]:
ddl_geq_cols = ['ddl_geq_' + str(int(i)) for i in ddl_range]
ddl_track_cols = ['ddl_track_' + str(int(i)) for i in ddl_range]

In [27]:
for i in range(len(ddl_range)):
    df_ebw_features[ddl_geq_cols[i]] = df_ebw_features['ddl_metric'].apply(lambda x: int(x >= ddl_range[i]))
    df_ebw_features[ddl_track_cols[i]] = df_ebw_features.groupby(['format_code', 'call_letters', 'mediabase_id'])[ddl_geq_cols[i]].cumsum()

### Isolate Hit songs

In [35]:
df_hits = df_ebw_features.join(df_ebw_features[df_ebw_features['num_top_quintile'] >=2].groupby(['format_code', 'call_letters', 'mediabase_id']).agg({'week_dt': np.min, 'weeks_since_release': np.min, 'cuml_spins_non_on': np.min}), how='right', on = ['format_code', 'call_letters', 'mediabase_id'], rsuffix='_hit')

In [37]:
df_hits_brn_h1 = df_hits.join(df_hits[(df_hits['ddl_track_20'] >= 2) & (df_hits['week_dt'] > df_hits['week_dt_hit'])].groupby(['format_code', 'call_letters', 'mediabase_id']).agg({'week_dt': np.min, 'weeks_since_release': np.min, 'cuml_spins_non_on':np.min}), how='left', on = ['format_code', 'call_letters', 'mediabase_id'], rsuffix='_brn')

In [39]:
df_hits_brn_h1[(df_hits_brn_h1['mediabase_id'] == 2436510) & (df_hits_brn_h1['call_letters'] == 'KBKS-FM')]

,mediabase_id,song_id,call_letters,format_code,week_dt,score_dt,song_release_date,last_callout_date,first_spin_date,market_name,...,cuml_spins_bucket,weeks_since_release,ddl_geq_20,ddl_track_20,week_dt_hit,weeks_since_release_hit,cuml_spins_non_on_hit,week_dt_brn,weeks_since_release_brn,cuml_spins_non_on_brn
4567,2436510,545826696,KBKS-FM,H1,2018-10-07,2018-10-15,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(0, 100]",0,0,0,2018-11-25,7,442.0,2019-04-28,29.0,2176.0
4568,2436510,545826696,KBKS-FM,H1,2018-10-14,2018-10-22,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(0, 100]",1,0,0,2018-11-25,7,442.0,2019-04-28,29.0,2176.0
4569,2436510,545826696,KBKS-FM,H1,2018-10-21,2018-10-29,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(0, 100]",2,0,0,2018-11-25,7,442.0,2019-04-28,29.0,2176.0
4570,2436510,545826696,KBKS-FM,H1,2018-10-28,2018-11-05,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(0, 100]",3,0,0,2018-11-25,7,442.0,2019-04-28,29.0,2176.0
4571,2436510,545826696,KBKS-FM,H1,2018-11-04,2018-11-12,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(100, 200]",4,0,0,2018-11-25,7,442.0,2019-04-28,29.0,2176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4787,2436510,545826696,KBKS-FM,H1,2022-12-25,2023-01-02,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(3300, 3400]",220,0,10,2018-11-25,7,442.0,2019-04-28,29.0,2176.0
4788,2436510,545826696,KBKS-FM,H1,2023-01-01,2023-01-09,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(3300, 3400]",221,0,10,2018-11-25,7,442.0,2019-04-28,29.0,2176.0
4789,2436510,545826696,KBKS-FM,H1,2023-01-08,2023-01-16,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(3300, 3400]",222,0,10,2018-11-25,7,442.0,2019-04-28,29.0,2176.0
4790,2436510,545826696,KBKS-FM,H1,2023-01-15,2023-01-23,2018-10-01,2021-12-13,2018-10-08,Seattle,...,"(3300, 3400]",223,0,10,2018-11-25,7,442.0,2019-04-28,29.0,2176.0


In [46]:
# Define lambas
fn_rolling_avg_4wk = lambda x: x.rolling(4, min_periods=1).mean()
fn_rolling_avg_8wk = lambda x: x.rolling(8, min_periods=1).mean()
fn_rolling_avg_13wk = lambda x: x.rolling(13, min_periods=1).mean()

fn_rolling_min_4wk = lambda x: x.rolling(4, min_periods=1).min()
fn_rolling_min_8wk = lambda x: x.rolling(8, min_periods=1).min()
fn_rolling_min_13wk = lambda x: x.rolling(13, min_periods=1).min()

fn_rolling_max_4wk = lambda x: x.rolling(4, min_periods=1).max()
fn_rolling_max_8wk = lambda x: x.rolling(8, min_periods=1).max()
fn_rolling_max_13wk = lambda x: x.rolling(13, min_periods=1).max()

In [63]:
df_grp = df_hits_brn_h1.groupby(['format_code', 'call_letters', 'mediabase_id'])

In [64]:
agg_cols = ['pop', 'ddl_metric', 'fav_metric', 'spins_non_on', 'spins_am_pm_dr', 'market_spins', 'format_spins']

In [ ]:
for col in agg_cols:
    df_hits_brn_h1[col + '_avg_4wk'] = df_grp[col].apply(fn_rolling_avg_4wk)
    df_hits_brn_h1[col + '_avg_8wk'] = df_grp[col].apply(fn_rolling_avg_8wk)
    df_hits_brn_h1[col + '_avg_13wk'] = df_grp[col].apply(fn_rolling_avg_13wk)

    df_hits_brn_h1[col + '_min_4wk'] = df_grp[col].apply(fn_rolling_min_4wk)
    df_hits_brn_h1[col + '_min_8wk'] = df_grp[col].apply(fn_rolling_min_8wk)
    df_hits_brn_h1[col + '_min_13wk'] = df_grp[col].apply(fn_rolling_min_13wk)

    df_hits_brn_h1[col + '_max_4wk'] = df_grp[col].apply(fn_rolling_max_4wk)
    df_hits_brn_h1[col + '_max_8wk'] = df_grp[col].apply(fn_rolling_max_8wk)
    df_hits_brn_h1[col + '_max_13wk'] = df_grp[col].apply(fn_rolling_max_13wk)

In [68]:
df_hits_brn_h1

,mediabase_id,song_id,call_letters,format_code,week_dt,score_dt,song_release_date,last_callout_date,first_spin_date,market_name,...,market_spins_min_13wk,market_spins_max_4wk,market_spins_max_8wk,market_spins_max_13wk,format_spins_min_4wk,format_spins_min_8wk,format_spins_min_13wk,format_spins_max_4wk,format_spins_max_8wk,format_spins_max_13wk
4567,2436510,545826696,KBKS-FM,H1,2018-10-07,2018-10-15,2018-10-01,2021-12-13,2018-10-08,Seattle,...,23.0,23.0,23.0,23.0,405.0,405.0,405.0,405.0,405.0,405.0
4568,2436510,545826696,KBKS-FM,H1,2018-10-14,2018-10-22,2018-10-01,2021-12-13,2018-10-08,Seattle,...,23.0,61.0,61.0,61.0,405.0,405.0,405.0,680.0,680.0,680.0
4569,2436510,545826696,KBKS-FM,H1,2018-10-21,2018-10-29,2018-10-01,2021-12-13,2018-10-08,Seattle,...,23.0,71.0,71.0,71.0,405.0,405.0,405.0,950.0,950.0,950.0
4570,2436510,545826696,KBKS-FM,H1,2018-10-28,2018-11-05,2018-10-01,2021-12-13,2018-10-08,Seattle,...,23.0,71.0,71.0,71.0,405.0,405.0,405.0,950.0,950.0,950.0
4571,2436510,545826696,KBKS-FM,H1,2018-11-04,2018-11-12,2018-10-01,2021-12-13,2018-10-08,Seattle,...,23.0,107.0,107.0,107.0,649.0,405.0,405.0,1148.0,1148.0,1148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041928,2754544,2012746527,WZFT-FM,H1,2022-12-25,2023-01-02,2021-10-01,2022-08-22,2022-01-31,Baltimore,...,0.0,2.0,4.0,7.0,37.0,37.0,37.0,210.0,210.0,210.0
1041929,2754544,2012746527,WZFT-FM,H1,2023-01-01,2023-01-09,2021-10-01,2022-08-22,2022-01-31,Baltimore,...,0.0,5.0,5.0,7.0,37.0,37.0,37.0,210.0,210.0,210.0
1041930,2754544,2012746527,WZFT-FM,H1,2023-01-08,2023-01-16,2021-10-01,2022-08-22,2022-01-31,Baltimore,...,0.0,5.0,5.0,5.0,103.0,37.0,37.0,210.0,210.0,210.0
1041931,2754544,2012746527,WZFT-FM,H1,2023-01-15,2023-01-23,2021-10-01,2022-08-22,2022-01-31,Baltimore,...,0.0,5.0,5.0,5.0,103.0,37.0,37.0,210.0,210.0,210.0


In [69]:
df_hits_brn_h1['week_dt_brn_4wk_prior'] = df_hits_brn_h1['week_dt_brn'] - np.timedelta64(4, 'W')

In [70]:
df_hits_brn_h1[['week_dt_brn', 'week_dt_brn_4wk_prior']]

,week_dt_brn,week_dt_brn_4wk_prior
4567,2019-04-28,2019-03-31
4568,2019-04-28,2019-03-31
4569,2019-04-28,2019-03-31
4570,2019-04-28,2019-03-31
4571,2019-04-28,2019-03-31
...,...,...
1041928,NaT,NaT
1041929,NaT,NaT
1041930,NaT,NaT
1041931,NaT,NaT


In [72]:
df_train = df_hits_brn_h1[(df_hits_brn_h1['week_dt'] > df_hits_brn_h1['week_dt_hit']) & (df_hits_brn_h1['week_dt_hit'] <= df_hits_brn_h1['week_dt_brn_4wk_prior'])]

In [74]:
df_train['target'] = df_train.apply(lambda x: int(x['week_dt'] == x['week_dt_brn_4wk_prior']), axis=1)

In [76]:
df_train.groupby(['target']).count()

,mediabase_id,song_id,call_letters,format_code,week_dt,score_dt,song_release_date,last_callout_date,first_spin_date,market_name,...,market_spins_max_4wk,market_spins_max_8wk,market_spins_max_13wk,format_spins_min_4wk,format_spins_min_8wk,format_spins_min_13wk,format_spins_max_4wk,format_spins_max_8wk,format_spins_max_13wk,week_dt_brn_4wk_prior
target,,,,,,,,,,,,,,,,,,,,,
0,162245,162245,162245,162245,162245,162245,162245,162245,162245,162245,...,162245,162245,162245,162245,162245,162245,162245,162245,162245,162245
1,1209,1209,1209,1209,1209,1209,1209,1209,1209,1209,...,1209,1209,1209,1209,1209,1209,1209,1209,1209,1209


In [85]:
cat_cols = ['Market']
num_cols = agg_cols + list(df_train.columns[-65:-2])

In [83]:
num_cols

Index(['pop_avg_4wk', 'pop_avg_8wk', 'pop_avg_13wk', 'ddl_metric_avg_4wk',
       'ddl_metric_avg_8wk', 'ddl_metric_avg_13wk', 'fav_metric_avg_4wk',
       'fav_metric_avg_8wk', 'fav_metric_avg_13wk', 'spins_non_on_avg_4wk',
       'spins_non_on_avg_8wk', 'spins_non_on_avg_13wk',
       'spins_am_pm_dr_avg_4wk', 'spins_am_pm_dr_avg_8wk',
       'spins_am_pm_dr_avg_13wk', 'market_spins_avg_4wk',
       'market_spins_avg_8wk', 'market_spins_avg_13wk', 'format_spins_avg_4wk',
       'format_spins_avg_8wk', 'format_spins_avg_13wk', 'pop_min_4wk',
       'pop_min_8wk', 'pop_min_13wk', 'pop_max_4wk', 'pop_max_8wk',
       'pop_max_13wk', 'ddl_metric_min_4wk', 'ddl_metric_min_8wk',
       'ddl_metric_min_13wk', 'ddl_metric_max_4wk', 'ddl_metric_max_8wk',
       'ddl_metric_max_13wk', 'fav_metric_min_4wk', 'fav_metric_min_8wk',
       'fav_metric_min_13wk', 'fav_metric_max_4wk', 'fav_metric_max_8wk',
       'fav_metric_max_13wk', 'spins_non_on_min_4wk', 'spins_non_on_min_8wk',
       'spins_no